Aim - to build a portfolio optimiser


In [5]:
print("hello")

hello


In [10]:
# importing all needed libraries
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize

In [11]:
# choosing our tickers (stocks) to follow
# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

tickers = ['AAPL']

# choosing start and end date, taking today's date to be the end date and giving us 1500 days of stock data
end_date = datetime.today()
start_date = end_date - timedelta(days = 30)

Now, we want to create our dataframe. This will be a dataframe storing the close price of every ticker in our list on each day in our start to end range. The close price in the yfinance dataframe is actually the adjusted close price, which takes into accounts dividends etc to give us a better overall picture of the stock price movement


In [12]:
closing_prices = pd.DataFrame()
for ticker in tickers:
  data = yf.download(ticker, start = start_date, end = end_date, auto_adjust = True)
  closing_prices[ticker] = data["Close"]

# Now cleaning the data so that we can operate on it successfully

closing_prices = closing_prices.dropna(how='any')
closing_prices = closing_prices.drop_duplicates()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Next, we want to start understanding returns. We can either go for simple returns, calculating direct percentage returns from one day to the next, or we can take the logarithm of this. Here, we will use log returns, as it will allow us to aggregate returns over time more nicely. Also, taking logarithms will make our distribution appear normalised, which will be helpful for interpreting results

In [ ]:
log_returns = np.log((closing_prices / closing_prices.shift(1)).dropna())

# closing_prices.shift(1) shifts our datapoints down by 1, so that at the row for time t, we have the close price at time t-1.
# We drop na as we will get an NaN in the first row due to the shift

Now, we start performing our data analysis. The first thing we will want to do is to get information that will be useful for our optimizer.

In [ ]:
mean_returns = log_returns.mean() * 252         # Gives us an expected annual return on our stock
covariance_matrix = log_returns.cov() * 252     # Gives us covariances between stocks, can help us understand how assets move together

# We multiply by 252 to annualise the data - log_returns currently stores daily information

# We also introduce our risk free rate variable, which will be needed in the sharpe ratio calculation

risk_free_rate = 0.05

We now want to analyse a portfolio more specifically. We want to look at 3 things - Expected return of a portfolio, volatility and sharpe ratio

In [ ]:
def expected_portfolio_returns(weights, mean_returns):
  return np.dot(weights, mean_returns)

def volatility(weights, mean_returns, covariance_matrix):
  return np.sqrt(np.dot(weights.T,np.dot(covariance_matrix,weights))) #standard deviation

def sharpe_ratio(weights, mean_returns, covariance_matrix, risk_free_rate):
  return (expected_portfolio_returns(weights, mean_returns) - risk_free_rate)/volatility(weights, mean_returns, covariance_matrix)

Now, minimizing the negative sharpe ratio (to maximize the sharpe ratio)

In [ ]:
def neg_sharpe(weights, mean_returns, covariance_matrix, risk_free_rate):
    return -sharpe_ratio(weights, mean_returns, covariance_matrix, risk_free_rate)

max_stock_percentage = 0.4 # maximum percentage of our portfolio that can be put on a single stock, to ensure that we don't put all our money into one stock
num_of_stocks = len(tickers)
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = [(0,max_stock_percentage) for i in range(num_of_stocks)]

initial_weights = [1/num_of_stocks for i in range(num_of_stocks)]

optimised_results = minimize(neg_sharpe,initial_weights, args = (mean_returns, covariance_matrix, risk_free_rate), method = "SLSQP", bounds = bounds, constraints = constraints)
optimal_weights = optimised_results.x

Analysing the portfolio using these optimal weights

In [ ]:
print("Optimal weights for portfolio")
for ticker, weight in zip(tickers, optimal_weights):
  print(f"{ticker} weight: {weight:.3f}")

optimal_portfolio_return = expected_portfolio_returns(optimal_weights, mean_returns)
optimal_portfolio_volatility = volatility(optimal_weights, mean_returns, covariance_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, mean_returns, covariance_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.3f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.3f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.3f}")

In [ ]:
print("hello")